In [1]:
# Install Java 11 (Spark plays nicely with this in Colab)
!apt-get install -y openjdk-11-jdk-headless > /dev/null

import os

# Set JAVA_HOME *before* importing pyspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

# Just to verify Java is there
!java -version

# Install a recent PySpark compatible with Java 11
!pip install -q pyspark==3.5.1


openjdk version "11.0.28" 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import math

spark = SparkSession.builder.appName("smart_parking_ml").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

spark


In [3]:
input_path = "/content/parking_ml_ready.csv"  # adjust if different

df_raw = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(input_path)
)

df_raw.printSchema()
df_raw.show(5, truncate=False)


root
 |-- device_id: integer (nullable = true)
 |-- street_marker: string (nullable = true)
 |-- sign_plate_id: string (nullable = true)
 |-- target_occupied: integer (nullable = true)
 |-- hour: double (nullable = true)
 |-- dow: double (nullable = true)
 |-- is_weekend: integer (nullable = true)
 |-- duration_min: double (nullable = true)

+---------+-------------+-------------+---------------+----+---+----------+-------------------+
|device_id|street_marker|sign_plate_id|target_occupied|hour|dow|is_weekend|duration_min       |
+---------+-------------+-------------+---------------+----+---+----------+-------------------+
|23913    |1581S        |nan          |1              |4.0 |6.0|1         |0.6                |
|23913    |1581S        |3.0          |1              |16.0|6.0|1         |0.43333333333333335|
|23913    |1581S        |3.0          |1              |18.0|6.0|1         |0.23333333333333334|
|23913    |1581S        |99.0         |0              |9.0 |1.0|0         |79.2 

In [4]:
# Rename target_occupied -> label and basic cleaning
df = (
    df_raw
    .withColumnRenamed("target_occupied", "label")
)

# Make sure label is numeric (0/1) and drop rows with missing key fields
df = (
    df
    .withColumn("label", F.col("label").cast("double"))
    .dropna(subset=["label", "street_marker", "sign_plate_id", "hour", "dow", "duration_min"])
)

# Basic features
df = df.withColumn(
    "log_duration_min",
    F.log1p(F.col("duration_min"))
)

# Discrete hour bucket (for interpretability, if you want to inspect later)
df = df.withColumn(
    "hour_bucket",
    F.when((F.col("hour") >= 7) & (F.col("hour") <= 10), F.lit(1))
     .when((F.col("hour") >= 16) & (F.col("hour") <= 19), F.lit(2))
     .otherwise(F.lit(0))
)

# Peak-hour flag
df = df.withColumn(
    "is_peak",
    F.when(F.col("hour_bucket") != 0, F.lit(1)).otherwise(F.lit(0))
)

print("Schema after basic cleaning & features:")
df.printSchema()

print("Label distribution:")
df.groupBy("label").count().orderBy("label").show()


Schema after basic cleaning & features:
root
 |-- device_id: integer (nullable = true)
 |-- street_marker: string (nullable = true)
 |-- sign_plate_id: string (nullable = true)
 |-- label: double (nullable = true)
 |-- hour: double (nullable = true)
 |-- dow: double (nullable = true)
 |-- is_weekend: integer (nullable = true)
 |-- duration_min: double (nullable = true)
 |-- log_duration_min: double (nullable = true)
 |-- hour_bucket: integer (nullable = false)
 |-- is_peak: integer (nullable = false)

Label distribution:
+-----+-----+
|label|count|
+-----+-----+
|  0.0| 7500|
|  1.0| 7499|
+-----+-----+



In [5]:
# Start from cleaned df
df_fe = df

# 1) Cyclical encoding for HOUR (0–23)
df_fe = df_fe.withColumn(
    "hour_sin",
    F.sin(F.lit(2 * math.pi) * F.col("hour") / F.lit(24.0))
).withColumn(
    "hour_cos",
    F.cos(F.lit(2 * math.pi) * F.col("hour") / F.lit(24.0))
)

# 2) Cyclical encoding for DOW (0–6 or 1–7)
df_fe = df_fe.withColumn(
    "dow_sin",
    F.sin(F.lit(2 * math.pi) * F.col("dow") / F.lit(7.0))
).withColumn(
    "dow_cos",
    F.cos(F.lit(2 * math.pi) * F.col("dow") / F.lit(7.0))
)

# 3) Aggregate stats per street_marker
marker_stats = (
    df_fe.groupBy("street_marker")
    .agg(
        F.avg("duration_min").alias("avg_duration_per_marker"),
        F.avg("label").alias("occ_rate_per_marker")
    )
)

df_fe = df_fe.join(marker_stats, on="street_marker", how="left")

print("Schema AFTER feature engineering:")
df_fe.printSchema()

df_fe.select(
    "street_marker", "hour", "dow",
    "hour_sin", "hour_cos", "dow_sin", "dow_cos",
    "avg_duration_per_marker", "occ_rate_per_marker", "label"
).show(5, truncate=False)


Schema AFTER feature engineering:
root
 |-- street_marker: string (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- sign_plate_id: string (nullable = true)
 |-- label: double (nullable = true)
 |-- hour: double (nullable = true)
 |-- dow: double (nullable = true)
 |-- is_weekend: integer (nullable = true)
 |-- duration_min: double (nullable = true)
 |-- log_duration_min: double (nullable = true)
 |-- hour_bucket: integer (nullable = false)
 |-- is_peak: integer (nullable = false)
 |-- hour_sin: double (nullable = true)
 |-- hour_cos: double (nullable = true)
 |-- dow_sin: double (nullable = true)
 |-- dow_cos: double (nullable = true)
 |-- avg_duration_per_marker: double (nullable = true)
 |-- occ_rate_per_marker: double (nullable = true)

+-------------+----+---+-------------------+-----------------------+-------------------+------------------+-----------------------+-------------------+-----+
|street_marker|hour|dow|hour_sin           |hour_cos               |dow_sin  

In [6]:
df_model = df_fe


In [11]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# Categorical columns (treated as categories, not numbers)
cat_cols = ["street_marker", "sign_plate_id","device_id"]

indexers = [
    StringIndexer(
        inputCol=c,
        outputCol=f"{c}_idx",
        handleInvalid="keep"   # avoid errors on unseen categories
    )
    for c in cat_cols
]

# Numeric + engineered features
numeric_features = [
    "hour",
    "dow",
    "is_weekend",
    "duration_min",
    "log_duration_min",
    "is_peak",
    "hour_sin",
    "hour_cos",
    "dow_sin",
    "dow_cos",
    "avg_duration_per_marker",
    "occ_rate_per_marker",
]

assembler = VectorAssembler(
    inputCols=numeric_features + [f"{c}_idx" for c in cat_cols],
    outputCol="features_raw"
)

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=False,
    withStd=True
)

preprocess_pipeline = Pipeline(stages=indexers + [assembler, scaler])

preprocess_model = preprocess_pipeline.fit(df_model)
df_prepared = preprocess_model.transform(df_model)

df_prepared.select("features", "label").show(5, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                             |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|[0.6727401171714494,3.21262444817411,2.4250446719252436,0.005771956227670986,0.27406169860009

In [8]:
train_df, test_df = df_prepared.randomSplit([0.7, 0.3], seed=42)

print("Train rows:", train_df.count())
print("Test rows:", test_df.count())


Train rows: 10616
Test rows: 4383


In [29]:
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import DoubleType
import pyspark.sql.functions as F

# 🔹 (Optional but helpful) Cache to speed up and reduce recomputation
train_df = train_df.cache()
test_df = test_df.cache()

# ============================================================
# 1️⃣ Tuned Random Forest (your original code)
# ============================================================

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    seed=42,
    featureSubsetStrategy="sqrt",  # common best practice
    probabilityCol="rf_prob"       # <-- important for stacking
)

# We will optimize for AUC
evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# 🔹 Lightweight param grid (ONLY 4 combinations)
paramGrid = (
    ParamGridBuilder()
      .addGrid(rf.numTrees, [100, 200])   # 2 options
      .addGrid(rf.maxDepth, [8, 12])      # 2 options
      .build()                            # 2 * 2 = 4 total
)

# 🔹 TrainValidationSplit instead of CrossValidator
tvs = TrainValidationSplit(
    estimator=rf,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio=0.8,   # 80% of train_df used for training each model, 20% for validation
    parallelism=1     # keep this 1 to reduce memory pressure
)

# 🚀 Fit the Tuning
tvs_model = tvs.fit(train_df)
best_rf_model = tvs_model.bestModel

print("Best Random Forest params:")
print("  numTrees:", best_rf_model.getOrDefault("numTrees"))
print("  maxDepth:", best_rf_model.getOrDefault("maxDepth"))

# 🔹 Evaluate RF on the held-out test_df
rf_preds = best_rf_model.transform(test_df)

auc_eval = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)
acc_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)
f1_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

auc = auc_eval.evaluate(rf_preds)
acc = acc_eval.evaluate(rf_preds)
f1 = f1_eval.evaluate(rf_preds)

print("\n==== Tuned Random Forest (TrainValidationSplit) ====")
print(f"AUC:      {auc:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"F1:       {f1:.4f}")

print("\nConfusion matrix (prediction vs label):")
rf_preds.groupBy("prediction", "label").count().orderBy("prediction", "label").show()

# ============================================================
# 2️⃣ Second base model: Logistic Regression
# ============================================================

lr_base = LogisticRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    regParam=0.01,
    probabilityCol="lr_prob",
    rawPredictionCol="lr_raw",
    predictionCol="lr_pred"
)

lr_base_model = lr_base.fit(train_df)

# ============================================================
# 3️⃣ Build level-1 (stacking) training data
#     Features for meta-learner = [RF_prob1, LR_prob1]
# ============================================================

# Apply both base models on train_df
train_lvl1 = best_rf_model.transform(train_df)      # adds rf_prob
train_lvl1 = lr_base_model.transform(train_lvl1)    # adds lr_prob

# UDF to extract P(class=1) from probability vector
vec_to_prob1_udf = F.udf(lambda v: float(v[1]), DoubleType())

train_lvl1 = (
    train_lvl1
        .withColumn("rf_prob1", vec_to_prob1_udf("rf_prob"))
        .withColumn("lr_prob1", vec_to_prob1_udf("lr_prob"))
)

# Assemble stacking features
stack_assembler = VectorAssembler(
    inputCols=["rf_prob1", "lr_prob1"],
    outputCol="stack_features"
)

train_lvl1 = stack_assembler.transform(train_lvl1)

# Meta-learner on top of RF + LR probabilities
stack_lr = LogisticRegression(
    featuresCol="stack_features",
    labelCol="label",
    maxIter=50,
    regParam=0.0,
    probabilityCol="stack_prob",
    rawPredictionCol="stack_raw",
    predictionCol="stack_pred"
)

stack_model = stack_lr.fit(train_lvl1)

# ============================================================
# 4️⃣ Apply stacking model on test set
# ============================================================

test_lvl1 = best_rf_model.transform(test_df)
test_lvl1 = lr_base_model.transform(test_lvl1)

test_lvl1 = (
    test_lvl1
        .withColumn("rf_prob1", vec_to_prob1_udf("rf_prob"))
        .withColumn("lr_prob1", vec_to_prob1_udf("lr_prob"))
)

test_lvl1 = stack_assembler.transform(test_lvl1)

stack_preds = stack_model.transform(test_lvl1)

# ============================================================
# 5️⃣ Evaluate stacking model
# ============================================================

stack_auc_eval = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="stack_raw",
    metricName="areaUnderROC"
)
stack_acc_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="stack_pred",
    metricName="accuracy"
)
stack_f1_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="stack_pred",
    metricName="f1"
)

stack_auc = stack_auc_eval.evaluate(stack_preds)
stack_acc = stack_acc_eval.evaluate(stack_preds)
stack_f1 = stack_f1_eval.evaluate(stack_preds)

print("\n==== Stacking Model (meta LR on [RF_prob1, LR_prob1]) ====")
print(f"AUC:      {stack_auc:.4f}")
print(f"Accuracy: {stack_acc:.4f}")
print(f"F1:       {stack_f1:.4f}")

print("\nStacking confusion matrix (stack_pred vs label):")
stack_preds.groupBy("stack_pred", "label").count().orderBy("stack_pred", "label").show()


Best Random Forest params:
  numTrees: 100
  maxDepth: 12

==== Tuned Random Forest (TrainValidationSplit) ====
AUC:      0.7968
Accuracy: 0.7157
F1:       0.7158

Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1555|
|       0.0|  1.0|  646|
|       1.0|  0.0|  603|
|       1.0|  1.0| 1590|
+----------+-----+-----+


==== Stacking Model (meta LR on [RF_prob1, LR_prob1]) ====
AUC:      0.7795
Accuracy: 0.7000
F1:       0.7001

Stacking confusion matrix (stack_pred vs label):
+----------+-----+-----+
|stack_pred|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1535|
|       0.0|  1.0|  695|
|       1.0|  0.0|  623|
|       1.0|  1.0| 1541|
+----------+-----+-----+



In [15]:
#RF Tuned

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import pyspark.sql.functions as F

# 🔹 (Optional but helpful) Cache to speed up and reduce recomputation
train_df = train_df.cache()
test_df = test_df.cache()

# Base Random Forest (we will tune only a couple of params)
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    seed=42,
    featureSubsetStrategy="sqrt"  # common best practice
)

# We will optimize for AUC
evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# 🔹 Lightweight param grid (ONLY 4 combinations)
paramGrid = (
    ParamGridBuilder()
      .addGrid(rf.numTrees, [100, 200])   # 2 options
      .addGrid(rf.maxDepth, [8, 12])      # 2 options
      .build()                            # 2 * 2 = 4 total
)

# 🔹 TrainValidationSplit instead of CrossValidator
tvs = TrainValidationSplit(
    estimator=rf,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio=0.8,   # 80% of train_df used for training each model, 20% for validation
    parallelism=1     # keep this 1 to reduce memory pressure
)

# 🚀 Fit the Tuning
tvs_model = tvs.fit(train_df)
best_rf_model = tvs_model.bestModel

print("Best Random Forest params:")
print("  numTrees:", best_rf_model.getOrDefault("numTrees"))
print("  maxDepth:", best_rf_model.getOrDefault("maxDepth"))

# 🔹 Evaluate on the held-out test_df (never seen in training/validation)
rf_preds = best_rf_model.transform(test_df)

auc_eval = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)
acc_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)
f1_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

auc = auc_eval.evaluate(rf_preds)
acc = acc_eval.evaluate(rf_preds)
f1 = f1_eval.evaluate(rf_preds)

print("\n==== Tuned Random Forest (TrainValidationSplit) ====")
print(f"AUC:      {auc:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"F1:       {f1:.4f}")

print("\nConfusion matrix (prediction vs label):")
rf_preds.groupBy("prediction", "label").count().orderBy("prediction", "label").show()



Best Random Forest params:
  numTrees: 200
  maxDepth: 12

==== Tuned Random Forest (TrainValidationSplit) ====
AUC:      0.7350
Accuracy: 0.6596
F1:       0.6596

Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1425|
|       0.0|  1.0|  733|
|       1.0|  0.0|  759|
|       1.0|  1.0| 1466|
+----------+-----+-----+



In [9]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

def train_and_eval(estimator, name: str):
    print(f"\n==== {name} ====")
    model = estimator.fit(train_df)
    preds = model.transform(test_df)

    # AUC
    auc_eval = BinaryClassificationEvaluator(
        labelCol="label",
        rawPredictionCol="rawPrediction",
        metricName="areaUnderROC"
    )
    auc = auc_eval.evaluate(preds)

    # Accuracy & F1 (multi-class evaluator works fine for binary)
    acc_eval = MulticlassClassificationEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="accuracy"
    )
    f1_eval = MulticlassClassificationEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="f1"
    )

    acc = acc_eval.evaluate(preds)
    f1 = f1_eval.evaluate(preds)

    print(f"AUC:      {auc:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1:       {f1:.4f}")

    print("Confusion matrix (prediction vs label):")
    preds.groupBy("prediction", "label").count().orderBy("prediction", "label").show()

    return model, preds


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    regParam=0.01,
    elasticNetParam=0.0
)

lr_model, lr_preds = train_and_eval(lr, "Logistic Regression")



==== Logistic Regression ====
AUC:      0.5829
Accuracy: 0.5494
F1:       0.5491
Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1147|
|       0.0|  1.0|  938|
|       1.0|  0.0| 1037|
|       1.0|  1.0| 1261|
+----------+-----+-----+



In [10]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=10,
    minInstancesPerNode=5,
    seed=42
)

rf_model, rf_preds = train_and_eval(rf, "Random Forest")



==== Random Forest ====
AUC:      0.7297
Accuracy: 0.6623
F1:       0.6623
Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1416|
|       0.0|  1.0|  712|
|       1.0|  0.0|  768|
|       1.0|  1.0| 1487|
+----------+-----+-----+



In [35]:
# Fix for 'JavaPackage' object is not callable error after installing SynapseML

# Stop existing SparkSession if running
if 'spark' in locals() and spark is not None:
    spark.stop()

# Re-initialize SparkSession to ensure SynapseML jars are picked up
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("smart_parking_ml").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

print("SparkSession restarted.")

# Re-run the data preparation steps to get df_prepared, train_df, and test_df
# as existing DataFrames will be linked to the old SparkSession.

# Reload the raw data
input_path = "/content/smart_parking_clean.csv"
df_full = (
    spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(input_path)
)

import pyspark.sql.functions as F

df_full = df_full.withColumnRenamed("target_occupied", "label")

# Drop leakage columns
leaky_cols = ["Status", "_occupied", "_occupied_imp"]
for c in leaky_cols:
    if c in df_full.columns:
        df_full = df_full.drop(c)

numeric_features = [
    "Hour", "DayOfWeek", "IsWeekend", "Month", "DayOfMonth",
    "DwellMinutes", "InViolation",
    "_duration_min", "_hour", "_dow", "_is_weekend",
    "_duration_min_w", "_duration_robust_z",
    "rolling_occ_N10", "arrivals_N10"
]

cat_features = [
    "PartOfDay", "AreaName", "SideName", "SideOfStreetCode",
    "street_marker_lumped", "device_id_lumped"
]

df_full = df_full.dropna(subset=["label"] + numeric_features + cat_features)
df_full = df_full.withColumn("label", F.col("label").cast("double"))

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

indexers = [
    StringIndexer(
        inputCol=c,
        outputCol=f"{c}_idx",
        handleInvalid="keep"
    )
    for c in cat_features
]

assembler = VectorAssembler(
    inputCols=numeric_features + [f"{c}_idx" for c in cat_features],
    outputCol="features_raw"
)

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=False,
    withStd=True
)

pipeline = Pipeline(stages=indexers + [assembler, scaler])
preprocess_model = pipeline.fit(df_full)
df_prepared = preprocess_model.transform(df_full)

train_df, test_df = df_prepared.randomSplit([0.7, 0.3], seed=42)

print("DataFrames 'train_df' and 'test_df' re-prepared successfully.")
print("Please re-run the subsequent cells, starting with the LightGBM training.")

SparkSession restarted.
DataFrames 'train_df' and 'test_df' re-prepared successfully.
Please re-run the subsequent cells, starting with the LightGBM training.


In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(
    featuresCol="features",
    labelCol="label",
    maxIter=80,
    maxDepth=5,
    stepSize=0.1,
    seed=42
)

gbt_model, gbt_preds = train_and_eval(gbt, "Gradient-Boosted Trees")



==== Gradient-Boosted Trees ====
AUC:      0.7265
Accuracy: 0.6616
F1:       0.6616
Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1430|
|       0.0|  1.0|  729|
|       1.0|  0.0|  754|
|       1.0|  1.0| 1470|
+----------+-----+-----+



In [ ]:
from pyspark.ml.classification import LinearSVC

svc = LinearSVC(
    featuresCol="features",
    labelCol="label",
    maxIter=50,
    regParam=0.1
)

svc_model, svc_preds = train_and_eval(svc, "Linear SVM (LinearSVC)")



==== Linear SVM (LinearSVC) ====
AUC:      0.5842
Accuracy: 0.5530
F1:       0.5529
Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1173|
|       0.0|  1.0|  948|
|       1.0|  0.0| 1011|
|       1.0|  1.0| 1251|
+----------+-----+-----+



In [16]:
input_path = "/content/smart_parking_clean.csv"

df_full = (
    spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(input_path)
)

df_full.printSchema()
df_full.show(5, truncate=False)


root
 |-- BayId: integer (nullable = true)
 |-- DeviceId: integer (nullable = true)
 |-- StreetMarker: string (nullable = true)
 |-- StreetName: string (nullable = true)
 |-- Sign: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- ArrivalTime: timestamp (nullable = true)
 |-- DepartureTime: timestamp (nullable = true)
 |-- DurationSeconds: double (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- IsWeekend: boolean (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayOfMonth: integer (nullable = true)
 |-- PartOfDay: string (nullable = true)
 |-- DwellMinutes: double (nullable = true)
 |-- AreaName: string (nullable = true)
 |-- StreetId: integer (nullable = true)
 |-- InViolation: boolean (nullable = true)
 |-- SignPlateID: string (nullable = true)
 |-- SideName: string (nullable = true)
 |-- SideOfStreet: integer (nullable = true)
 |-- SideOfStreetCode: string (nullable = true)
 |-- _duration_min: double 

In [17]:
import pyspark.sql.functions as F

df_full = df_full.withColumnRenamed("target_occupied", "label")

# Drop leakage columns (they are EXACTLY equal to the label)
leaky_cols = ["Status", "_occupied", "_occupied_imp"]

for c in leaky_cols:
    if c in df_full.columns:
        df_full = df_full.drop(c)

numeric_features = [
    "Hour", "DayOfWeek", "IsWeekend", "Month", "DayOfMonth",
    "DwellMinutes", "InViolation",
    "_duration_min", "_hour", "_dow", "_is_weekend",
    "_duration_min_w", "_duration_robust_z",
    "rolling_occ_N10", "arrivals_N10"
]

cat_features = [
    "PartOfDay", "AreaName", "SideName", "SideOfStreetCode",
    "street_marker_lumped", "device_id_lumped"
]

# Drop rows with nulls
df_full = df_full.dropna(subset=["label"] + numeric_features + cat_features)

df_full = df_full.withColumn("label", F.col("label").cast("double"))


In [18]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

indexers = [
    StringIndexer(
        inputCol=c,
        outputCol=f"{c}_idx",
        handleInvalid="keep"
    )
    for c in cat_features
]

assembler = VectorAssembler(
    inputCols=numeric_features + [f"{c}_idx" for c in cat_features],
    outputCol="features_raw"
)

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=False,
    withStd=True
)

pipeline = Pipeline(stages=indexers + [assembler, scaler])
preprocess_model = pipeline.fit(df_full)
df_prepared = preprocess_model.transform(df_full)

train_df, test_df = df_prepared.randomSplit([0.7, 0.3], seed=42)


In [30]:
# FROM NEW DATASET

input_path = "/content/smart_parking_clean.csv"

df_full = (
    spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv(input_path)
)

import pyspark.sql.functions as F

df_full = df_full.withColumnRenamed("target_occupied", "label")

# Drop leakage columns
leaky_cols = ["Status", "_occupied", "_occupied_imp"]
for c in leaky_cols:
    if c in df_full.columns:
        df_full = df_full.drop(c)

numeric_features = [
    "Hour", "DayOfWeek", "IsWeekend", "Month", "DayOfMonth",
    "DwellMinutes", "InViolation",
    "_duration_min", "_hour", "_dow", "_is_weekend",
    "_duration_min_w", "_duration_robust_z",
    "rolling_occ_N10", "arrivals_N10"
]

cat_features = [
    "PartOfDay", "AreaName", "SideName", "SideOfStreetCode",
    "street_marker_lumped", "device_id_lumped"
]

df_full = df_full.dropna(subset=["label"] + numeric_features + cat_features)
df_full = df_full.withColumn("label", F.col("label").cast("double"))


In [31]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

indexers = [
    StringIndexer(
        inputCol=c,
        outputCol=f"{c}_idx",
        handleInvalid="keep"
    )
    for c in cat_features
]

assembler = VectorAssembler(
    inputCols=numeric_features + [f"{c}_idx" for c in cat_features],
    outputCol="features_raw"
)

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=False,
    withStd=True
)

pipeline = Pipeline(stages=indexers + [assembler, scaler])
preprocess_model = pipeline.fit(df_full)
df_prepared = preprocess_model.transform(df_full)

train_df, test_df = df_prepared.randomSplit([0.7, 0.3], seed=42)


In [32]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import pyspark.sql.functions as F

# Cache to reduce recomputation
train_df = train_df.cache()
test_df = test_df.cache()

# Base Random Forest (same as before)
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    seed=42,
    featureSubsetStrategy="sqrt"
)

# Optimize for AUC
evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

# Small, safe param grid (4 combinations)
paramGrid = (
    ParamGridBuilder()
      .addGrid(rf.numTrees, [100, 200])
      .addGrid(rf.maxDepth, [8, 12])
      .build()
)

# TrainValidationSplit (lighter than CrossValidator)
tvs = TrainValidationSplit(
    estimator=rf,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio=0.8,
    parallelism=1    # important for Colab memory
)

# Fit on the NEW richer train_df
tvs_model = tvs.fit(train_df)
best_rf_model = tvs_model.bestModel

print("Best Random Forest params:")
print("  numTrees:", best_rf_model.getOrDefault("numTrees"))
print("  maxDepth:", best_rf_model.getOrDefault("maxDepth"))

# Evaluate on test set
rf_preds = best_rf_model.transform(test_df)

auc_eval = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)
acc_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)
f1_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

auc = auc_eval.evaluate(rf_preds)
acc = acc_eval.evaluate(rf_preds)
f1 = f1_eval.evaluate(rf_preds)

print("\n==== Tuned Random Forest (Rich Feature Set) ====")
print(f"AUC:      {auc:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"F1:       {f1:.4f}")

print("\nConfusion matrix (prediction vs label):")
rf_preds.groupBy("prediction", "label").count().orderBy("prediction", "label").show()


Best Random Forest params:
  numTrees: 100
  maxDepth: 12

==== Tuned Random Forest (Rich Feature Set) ====
AUC:      0.7968
Accuracy: 0.7157
F1:       0.7158

Confusion matrix (prediction vs label):
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1555|
|       0.0|  1.0|  646|
|       1.0|  0.0|  603|
|       1.0|  1.0| 1590|
+----------+-----+-----+



In [37]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

gbt = GBTClassifier(
    featuresCol="features",
    labelCol="label",
    maxIter=80,       # strong but safe number
    maxDepth=8,       # usually best for GBT
    stepSize=0.1,     # learning rate
    seed=42
)

gbt_model = gbt.fit(train_df)
gbt_preds = gbt_model.transform(test_df)

auc_eval = BinaryClassificationEvaluator(
    labelCol="label",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)
acc_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)
f1_eval = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="f1"
)

auc = auc_eval.evaluate(gbt_preds)
acc = acc_eval.evaluate(gbt_preds)
f1 = f1_eval.evaluate(gbt_preds)

print("\n==== Gradient Boosted Trees ====")
print(f"AUC:      {auc:.4f}")
print(f"Accuracy: {acc:.4f}")
print(f"F1:       {f1:.4f}")

gbt_preds.groupBy("prediction", "label").count().orderBy("prediction", "label").show()



==== Gradient Boosted Trees ====
AUC:      0.7827
Accuracy: 0.7003
F1:       0.7003
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0| 1526|
|       0.0|  1.0|  685|
|       1.0|  0.0|  632|
|       1.0|  1.0| 1551|
+----------+-----+-----+

